In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# https://docs.python.org/3/library/io.html
import io
import os

# https://python-guide-pt-br.readthedocs.io/fr/latest/scenarios/imaging.html
from PIL import Image 

# https://docs.python.org/fr/3/library/hashlib.html
import hashlib # permet de chiffrer 

import csv
import time
import random
import sys
import numpy as np
 
# selenium package
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.common.exceptions import WebDriverException
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
import multiprocessing
from threading import Thread
from time import sleep

# beautiful soup
from bs4 import BeautifulSoup 
import requests 

# image traitement et features extraction
from PIL import Image, ImageOps, ImageEnhance
import os, sys
import numpy as np
import mahotas
import cv2
import os
import h5py

from time import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture 
from sklearn import mixture
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import multivariate_normal
from matplotlib import cm
from matplotlib.colors import LogNorm
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.decomposition import PCA

from sklearn.cluster import KMeans
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### SVM
from sklearn import neighbors, datasets, preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score, precision_score, recall_score, r2_score, mean_absolute_error, mean_squared_error, mean_absolute_error
##SVM
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline


from mlxtend.preprocessing import shuffle_arrays_unison

import pickle

In [ ]:
####################################
images_per_class = 80
fixed_size = tuple((56, 56))
MAINDIR='C:/Users/utilisateur/Documents/DataIA/Projet/Projet_Classification_ZARA/Algo/'
DATADIR=MAINDIR + 'images_without_models/' # répertoire contenant les images réparties  dans les dossiers categories
CATEGORIES=os.listdir(DATADIR) #ie : jupes, pantalons, robes, t-shirts (attention j'ai un s à la fin des termes)
image_test='C:/Users/utilisateur/Documents/DataIA/Projet/Projet_Classification_ZARA/Algo/Images/jupes/jupe_0.jpg'
####################################



############### FONCTION POUR RENOMMER LES IMAGES en "category"_xx.jpg #########################
def rename_file(CATECORIES):
    for category in CATECORIES:
        path=DATADIR+category
        for i, filename in enumerate(os.listdir(path)):
            os.rename(path + "/" + filename, path + "/" + category[:-1]+'_'+ str(i) + ".jpg") # folder[:-1] c'est juste pour enlever le s
            # output = jupe_0.jpg ; jupe_1.jpg etc...

################# traitement d'une image ###################
def traitement_image(image_path):
    ### Load image ###
    img = cv2.imread(image_path)
#   plt.imshow(img)
#   plt.show()

    ### resize ###
    resized=cv2.resize(img, fixed_size, interpolation=cv2.INTER_AREA)
#   plt.imshow(resized)
#   plt.show()

    ### convert the image to grayscale ###
    imgGray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)  
#   cv2.imshow('title',imgGray)
#   cv2.waitKey(0)
#   cv2.destroyAllWindows()
    return imgGray


############### CREATION D'un Dataset à partir des images traitées ########################
Data = [] # Data contient la matrice des pixels de l'image en ligne => 1 lignes = 1 images (par ex Image[0] : 1x(3136) si on fait resize 56par56 )
Target_name = [] # Target_Name C'est la classe de chaque image sous forme ('jupe', 'robe', 't-shirt', 'pantalon')
Data_name = [] # Data_name contient le nom de l'image  par exemple : jupe_01
Image=[] #Image est une matrice de matrice => 1 ligne = 1 images en format matrice (par ex  Image[0] : 1x56x56

def DataSet():
    for category in CATEGORIES: 
        clothe_class=category[:-1]  #ie jupe, pantalon, robe or t-shirt
        for image in os.listdir(DATADIR+category):  #image : nom de l'image ie jupe_0.jpg
            image_path =os.path.join(DATADIR+category+'/'+image) # path = path de l'image
            imgGray=traitement_image(image_path)
            imgGray_flat=np.array(imgGray).flatten()
            try:
                Image.append(imgGray)
                Target_name.append(clothe_class)
                Data.append(imgGray_flat)
                Data_name.append(image[:-4])
            except Exception as e:
                print(e)
    return np.array(Data), np.array(Target_name), np.array(Data_name), np.array(Image) 

def DataSetCreation(DataSet_name):
    ## on crée le data set
    Data, Target_name, Data_Name, Image = DataSet()  
    
    ## vérification des dimensions 
    print(Data.shape, Target_name.shape,Data_Name.shape,Image.shape)
    
    ### mélange des données avec shuffle ; unison c'est pour que tous les arrays soient mélanger de la même façon et q'uon garde la correspondace entre eux
    Data, Target_name, Data_Name, Image = shuffle_arrays_unison(arrays=[Data, Target_name, Data_Name, Image], random_seed=0)
   
    ### encodage  pantalon to 1 ; jupe to 0 ;  t-shirt to 3 and robe to 2 pour chaque images 
    
    Target=pd.Series(Target_name).astype('category').cat.codes
    #print(Target)
    
    #### création d'un dictionnaire pour savoir la correspondance code <-> vetement 
    Target_name_list = dict(enumerate(pd.Series(Target_name).astype('category').cat.categories)) # ie {0: 'jupe', 1: 'pantalon', 2: 'robe', 3: 't-shirt'}
#   print(Target_name_list) 
    ### Creation du data set sous forme d'un dictionnaire comme quand on load les dataset scikitlearn
    DataSet_name={'data': Data,
                        'target':Target,
                        'target_names':Target_name,
                        'target_name_list':list(Target_name_list.values()),
                        'data_Name':Data_Name,
                        'images': Image
                       }
    ### pour sauver le dataset et pouvoir le reloader plus tard dans un autre notebook
    with open(str(DataSet_name)+".pickle","wb") as outpout:
        pickle.dump(DataSet_name, outpout)
        
    return DataSet_name
    # # then you can load it back in another script.
    # with open('ZARA_DataSet_wo_models.pickle', 'rb') as data:
    #     ZARA_DataSet_wo_models = pickle.load(data)

In [ ]:
ZARA_DataSet_wo_models=DataSetCreation(ZARA_DataSet_wo_models)

In [ ]:
ZARA_DataSet_wo_models.keys()
# Pour pouvoir utiliser les differents data du dataset sans avoir à écrire le nom du dataset à chaque fois 
for key in ZARA_DataSet_wo_models.keys():
    globals()[str(key)] =ZARA_DataSet_wo_models[key]
